In [ ]:
#working from gpuserver0
#venv used
#ts_env                   /home/shuhangli/miniconda3/envs/ts_env
from prometheus_api_client import PrometheusConnect, MetricsList, Metric, MetricRangeDataFrame
from prometheus_api_client.utils import parse_datetime
from datetime import datetime, timedelta
import pprint
import pandas as pd
pd.set_option('display.max_columns', None)  # or 1000
import argparse

import optuna
from optuna.integration import PyTorchLightningPruningCallback
from optuna.visualization import (
    plot_optimization_history,
    plot_contour,
    plot_param_importances,
)
import torch
import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler 

from darts.datasets import ElectricityDataset
from darts.models import TCNModel, LinearRegressionModel, NBEATSModel, TiDEModel, TSMixerModel
from darts.dataprocessing.transformers import Scaler
from darts.metrics import smape, mape, rmse, mae 
from darts.utils.likelihood_models import GaussianLikelihood, QuantileRegression
from darts import TimeSeries
from darts.dataprocessing.transformers import (
    Scaler,
    MissingValuesFiller,
    Mapper,
    InvertibleMapper,
)
from darts.metrics import mape

In [ ]:
df = pd.read_csv('/home/shuhangli/tstest/GEM1h.csv', parse_dates=True, index_col='Time')
#remove  µA from the data and convert to float
for column in df.columns:
    # Check if the column data type is object, indicating it might contain strings
    if df[column].dtype == 'object':
        # Remove 'µA' and convert to float
        df[column] = df[column].str.replace('µA', '').str.strip().astype(float)

data_min = df.max().max()
data_max = df.min().min()
print(data_min, data_max)
df.head()


In [ ]:

df_HV = pd.read_csv('/home/shuhangli/tstest/GEMHV1h.csv', parse_dates=True, index_col='Time')
df_HV.columns = df_HV.columns.str.replace('u', 'u_HV_')
for column in df_HV.columns:
    if df_HV[column].dtype == 'object':
        df_HV[column] = df_HV[column].str.replace('kV', '').str.strip().astype(float)
df_HV.head()


In [ ]:
df_CM = pd.read_csv('/home/shuhangli/tstest/CM_VI1h.csv', parse_dates=True, index_col='Time')
df_CM['Voltage'] = df_CM['Voltage'].str.replace('kV', '').str.strip().astype(float)
df_CM['Current'] = df_CM['Current'].str.replace('µA', '').str.strip().astype(float)
df_CM.head()

In [ ]:
df_CAD = pd.read_csv('/home/shuhangli/tstest/CAD1h.csv', parse_dates=True, index_col='Time')

def convert_units(column):
    if df_CAD[column].dtype == 'object':
        # Remove the unit and convert to float
        df_CAD[column] = df_CAD[column].str.replace('kHz', '').str.replace('Hz', '').str.strip().astype(float)
        # Convert kHz to Hz
        if 'kHz' in column:
            df_CAD[column] = df_CAD[column] * 1000
    return df_CAD[column]

# Process each column
for column in df_CAD.columns:
    df_CAD[column] = convert_units(column)

# Display the first few rows of the DataFrame
df_CAD.head()

In [ ]:
#joint
df_cov = df_HV.join(df_CM).join(df_CAD)
series_cov_raw = TimeSeries.from_dataframe(df_cov)
covfiller = MissingValuesFiller()
series_cov = covfiller.transform(series_cov_raw, method='quadratic')
Scaler_cov = Scaler(MinMaxScaler())
series_cov_scaled = Scaler_cov.fit_transform(series_cov)


In [ ]:
series_raw = TimeSeries.from_dataframe(df)

series_list_raw = []
for col in df.columns:
    ts = TimeSeries.from_dataframe(df, value_cols=col)
    series_list_raw.append(ts)

filler = MissingValuesFiller()
series = filler.transform(series_raw, method="quadratic")
Scaler_data = Scaler(MinMaxScaler())
series_scaled = Scaler_data.fit_transform(series)
#global_scaler = Mapper(lambda x: 1- (x-data_min)/(data_max - data_min) )
#series_scaled = global_scaler.transform(series)
#series_scaled.plot()


series_list = filler.transform(series_list_raw, method="quadratic")
Scaler_tslist = Scaler(scaler=MinMaxScaler(),global_fit=True, verbose=False)
#series_list_scaled = Scaler_tslist.fit_transform(series_list)
# sepereate the data into training and validation
length = len(series_list[0])
val_len = int(length/2)

series_list_scaled = Scaler_tslist.fit_transform(series_list)

train_series = [ts[: -(1 * val_len)] for ts in series_list_scaled]
val_series = [ts[-(1 * val_len): ] for ts in series_list_scaled]
test_series = [ts[-val_len:] for ts in series_list_scaled]

series_cov_scaled_list = [series_cov_scaled for i in range(len(series_list))]



#combine train_series to a multi-variate time series
train = series_scaled[: -(1 * val_len)]
val = series_scaled[-(1 * val_len): ]
test = series_scaled[-val_len:]

#test_scaler = Scaler()
#train_series[0] = test_scaler.fit_transform(train_series[0])
#val_series[0] = test_scaler.transform(val_series[0])
#test_series[0] = test_scaler.transform(test_series[0])
series_scaled.plot()
#print(train_series[0])
#train_series[0].plot()




In [ ]:
# some fixed parameters that will be the same for all models
BATCH_SIZE = 16
MAX_N_EPOCHS = 300
NR_EPOCHS_VAL_PERIOD = 1
MAX_SAMPLES_PER_TS = 1000

def build_fit_tcn_model(
    in_len,
    out_len,
    kernel_size,
    num_filters,
    weight_norm,
    dilation_base,
    dropout,
    lr,
    likelihood=None,
    callbacks=None,
):

    # reproducibility
    torch.manual_seed(42)


    # throughout training we'll monitor the validation loss for early stopping
    early_stopper = EarlyStopping("val_loss", min_delta=0.00, patience=10, verbose=True)
    if callbacks is None:
        callbacks = [early_stopper]
    else:
        callbacks = [early_stopper] + callbacks

    # detect if a GPU is available
    if torch.cuda.is_available():
        pl_trainer_kwargs = {
            "accelerator": "gpu",
            "devices": [0],
            #"gpus": -1,
            #"auto_select_gpus": True,
            "callbacks": callbacks,
        }
        num_workers = 4
    else:
        pl_trainer_kwargs = {"callbacks": callbacks}
        num_workers = 0

    encoders = None

    # build the TCN model
    model = TCNModel(
        input_chunk_length=in_len,
        output_chunk_length=out_len,
        batch_size=BATCH_SIZE,
        n_epochs=MAX_N_EPOCHS,
        nr_epochs_val_period=NR_EPOCHS_VAL_PERIOD,
        kernel_size=kernel_size,
        num_filters=num_filters,
        weight_norm=weight_norm,
        dilation_base=dilation_base,
        dropout=dropout,
        optimizer_kwargs={"lr": lr},
        add_encoders=encoders,
        likelihood=likelihood,
        pl_trainer_kwargs=pl_trainer_kwargs,
        model_name="tcn_model",
        force_reset=True,
        save_checkpoints=True,
    )


    model.fit( 
        series=train,
        val_series=val,
        max_samples_per_ts=MAX_SAMPLES_PER_TS,
        past_covariates=series_cov_scaled,
        val_past_covariates=series_cov_scaled,
        #num_loader_workers=num_workers,
    )

    # reload best model over course of training
    model = TCNModel.load_from_checkpoint("tcn_model")

    return model

def build_fit_nbeats_model(
    in_len,
    out_len,
    generic_architecture,
    num_stacks,
    num_blocks,
    num_layers,
    layer_widths,
    dropout,
    lr,
    likelihood=None,
    callbacks=None,
):

    # reproducibility
    torch.manual_seed(42)


    # throughout training we'll monitor the validation loss for early stopping
    early_stopper = EarlyStopping(monitor="val_loss", min_delta=0.00, patience=10, verbose=True)
    if callbacks is None:
        callbacks = [early_stopper]
    else:
        callbacks = [early_stopper] + callbacks

    # detect if a GPU is available
    if torch.cuda.is_available():
        pl_trainer_kwargs = {
            "accelerator": "gpu",
            "devices": [0],
            "callbacks": callbacks,
        }
        num_workers = 4
    else:
        pl_trainer_kwargs = {"callbacks": callbacks}
        num_workers = 0

    # build the NBEATS model
    model = NBEATSModel(
        input_chunk_length=in_len,
        output_chunk_length=out_len,
        generic_architecture=generic_architecture,
        num_stacks=num_stacks,
        num_blocks=num_blocks,
        num_layers=num_layers,
        layer_widths=layer_widths,
        batch_size=BATCH_SIZE,
        n_epochs=MAX_N_EPOCHS,
        nr_epochs_val_period=NR_EPOCHS_VAL_PERIOD,
        optimizer_kwargs={"lr": lr},
        likelihood=likelihood,
        dropout=dropout,
        pl_trainer_kwargs=pl_trainer_kwargs,
        model_name="nbeats_model",
        force_reset=True,
        save_checkpoints=True,
    )


    # train the model
    model.fit( 
        series=train,
        val_series=val,
        max_samples_per_ts=MAX_SAMPLES_PER_TS,
        past_covariates=series_cov_scaled,
        val_past_covariates=series_cov_scaled,
    )

    # reload best model over course of training
    model = NBEATSModel.load_from_checkpoint("nbeats_model")

    return model

def build_fit_tide_model(
    in_len,
    out_len,
    output_chunk_shift,
    num_encoder_layers,
    num_decoder_layers,
    decoder_output_dim,
    hidden_size,
    temporal_width_past,
    temporal_width_future,
    temporal_decoder_hidden,
    use_layer_norm,
    dropout,
    lr,
    likelihood=None,
    callbacks=None,
):

    # reproducibility
    torch.manual_seed(42)

    

    # throughout training we'll monitor the validation loss for early stopping
    early_stopper = EarlyStopping("val_loss", min_delta=0.00, patience=10, verbose=True)
    if callbacks is None:
        callbacks = [early_stopper]
    else:
        callbacks = [early_stopper] + callbacks

    # detect if a GPU is available
    if torch.cuda.is_available():
        pl_trainer_kwargs = {
            "accelerator": "gpu",
            "devices": [0],
            "callbacks": callbacks,
        }
        num_workers = 4
    else:
        pl_trainer_kwargs = {"callbacks": callbacks}
        num_workers = 0

    # build the TiDE model
    model = TiDEModel(
        input_chunk_length=in_len,
        output_chunk_length=out_len,
        output_chunk_shift=output_chunk_shift,
        num_encoder_layers=num_encoder_layers,
        num_decoder_layers=num_decoder_layers,
        decoder_output_dim=decoder_output_dim,
        hidden_size=hidden_size,
        temporal_width_past=temporal_width_past,
        temporal_width_future=temporal_width_future,
        temporal_decoder_hidden=temporal_decoder_hidden,
        use_layer_norm=use_layer_norm,
        dropout=dropout,
        use_static_covariates=False,  
        optimizer_kwargs={"lr": lr},
        likelihood=likelihood,
        pl_trainer_kwargs=pl_trainer_kwargs,
        model_name="tide_model",
        force_reset=True,
        save_checkpoints=True,
        n_epochs = MAX_N_EPOCHS,
    )

    # Train the model
    model.fit(
        series=train,
        val_series=val,
        max_samples_per_ts=MAX_SAMPLES_PER_TS,
        past_covariates=series_cov_scaled,
        val_past_covariates=series_cov_scaled,
    )

    # Reload best model over course of training
    model = TiDEModel.load_from_checkpoint("tide_model")

    return model

def build_fit_tsmixer_model(
    in_len,
    out_len,
    output_chunk_shift,
    hidden_size,
    ff_size,
    num_blocks,
    activation,
    dropout,
    norm_type,
    normalize_before,
    lr,
    likelihood=None,
    callbacks=None,
):

    # reproducibility
    torch.manual_seed(42)


    # early stopping
    early_stopper = EarlyStopping(monitor="val_loss", min_delta=0.00, patience=10, verbose=True)
    if callbacks is None:
        callbacks = [early_stopper]
    else:
        callbacks.append(early_stopper)

    if torch.cuda.is_available():
        pl_trainer_kwargs = {
            "accelerator": "gpu",
            "devices": [0],
            "callbacks": callbacks,
        }
        num_workers = 4
    else:
        pl_trainer_kwargs = {"callbacks": callbacks}
        num_workers = 0

    # build the TSMixer model
    model = TSMixerModel(
        input_chunk_length=in_len,
        output_chunk_length=out_len,
        output_chunk_shift=output_chunk_shift,
        hidden_size=hidden_size,
        ff_size=ff_size,
        num_blocks=num_blocks,
        activation=activation,
        dropout=dropout,
        norm_type=norm_type,
        normalize_before=normalize_before,
        use_static_covariates=False,
        optimizer_kwargs={"lr": lr},
        likelihood=likelihood,
        pl_trainer_kwargs=pl_trainer_kwargs,
        model_name="tsmixer_model",
        force_reset=True,
        save_checkpoints=True,
        n_epochs=MAX_N_EPOCHS,
    )

    # Train the model
    model.fit(
        series=train,
        val_series=val,
        max_samples_per_ts=MAX_SAMPLES_PER_TS,
        past_covariates=series_cov_scaled,
        val_past_covariates=series_cov_scaled,
    )

    # Reload best model over course of training
    model = TSMixerModel.load_from_checkpoint("tsmixer_model")

    return model

def eval_model(preds, name, train_set=train, val_set=val):
    smapes = smape(preds, val_set)
    print("{} sMAPE: {:.2f} +- {:.2f}".format(name, np.mean(smapes), np.std(smapes)))
    #mae and mse
    maes = mae(preds, val_set)
    print("{} MAE: {:.4f} +- {:.4f}".format(name, np.mean(maes), np.std(maes)))
    rmses = rmse(preds, val_set)
    print("{} RMSE: {:.4f} +- {:.4f}".format(name, np.mean(rmses), np.std(rmses)))

def eval_model_series(preds, name, train_set=train, val_set=val):
    smapes = smape(preds, val_set)
    print("{} sMAPE: {:.2f} +- {:.2f}".format(name, np.mean(smapes), np.std(smapes)))
    #mae and mse
    maes = mae(preds, val_set)
    print("{} MAE: {:.4f} +- {:.4f}".format(name, np.mean(maes), np.std(maes)))
    rmses = rmse(preds, val_set)
    print("{} RMSE: {:.4f} +- {:.4f}".format(name, np.mean(rmses), np.std(rmses)))


In [ ]:
lr_model = LinearRegressionModel(lags=60, 
                                 lags_past_covariates=10,
                                 output_chunk_length=3,
                                 )
lr_model.fit(train, 
             past_covariates=series_cov_scaled
             )
lr_preds = lr_model.predict(series=val[:60], n=60, 
                            past_covariates=series_cov_scaled,
                            num_samples=1)
historical_forecasts_lr = lr_model.historical_forecasts(series=val, retrain=False, past_covariates=series_cov_scaled, forecast_horizon=1)
eval_model(lr_preds, "LR model forecasts")

eval_model(historical_forecasts_lr, "LR model historical forecasts")

In [ ]:
model = build_fit_tcn_model(
    in_len=60,
    out_len=40,
    kernel_size=5,
    num_filters=5,
    weight_norm=False,
    dilation_base=3,
    dropout=0.1,
    lr=1e-3,
    #likelihood=QuantileRegression(),
)

In [ ]:
index = 13
preds = model.predict(series=train, n=120, num_samples=100)
eval_model(preds, "TCN", train_set=train, val_set=val)

In [ ]:

#loop over all the series
for i in range(10):
    plt.figure(figsize=(15, 5))
    Scaler_data.inverse_transform(train).univariate_component(i).plot()
    Scaler_data.inverse_transform(val).univariate_component(i).plot(label="actual")
    Scaler_data.inverse_transform(preds).univariate_component(i).plot(label="forecast")
    Scaler_data.inverse_transform(lr_preds).univariate_component(i).plot(label="LR forecast")
#eval_model(preds, "First TCN model")

In [ ]:
model_nbeats = build_fit_nbeats_model(
    in_len=60,
    out_len=30,
    generic_architecture=True,
    num_stacks=10,
    num_blocks=1,
    num_layers=4,
    layer_widths=256,
    dropout=0.1,
    lr=1e-3,
    likelihood=QuantileRegression(),
)

In [ ]:
preds = model_nbeats.predict(series=train, n=120, 
                             num_samples=1000
                             )
eval_model(preds, "NBEATS", train_set=train, val_set=val)

In [ ]:

#val['u303'].plot()
#preds['u303'].plot()
for i in range(10):
    plt.figure(figsize=(15, 5))
    Scaler_data.inverse_transform(train).univariate_component(i).plot()
    Scaler_data.inverse_transform(val).univariate_component(i).plot(label="actual")
    Scaler_data.inverse_transform(preds).univariate_component(i).plot(label="forecast")
    Scaler_data.inverse_transform(lr_preds).univariate_component(i).plot(label="LR forecast")
    

In [ ]:

for i in range(lr_preds.n_components):
    plt.figure(figsize=(15, 5))
    train.univariate_component(i).plot()
    val.univariate_component(i).plot(label="actual")
    lr_preds.univariate_component(i).plot(label="forecast")

In [ ]:
in_len = 60
out_len = 40
output_chunk_shift = 0
num_encoder_layers = 2
num_decoder_layers = 2
decoder_output_dim = 16
hidden_size = 128
temporal_width_past = 8
temporal_width_future = 8
temporal_decoder_hidden = 32
use_layer_norm = True
dropout = 0.1
lr = 1e-3

# Build and fit the model
model_tide = build_fit_tide_model(
    in_len=in_len,
    out_len=out_len,
    output_chunk_shift=output_chunk_shift,
    num_encoder_layers=num_encoder_layers,
    num_decoder_layers=num_decoder_layers,
    decoder_output_dim=decoder_output_dim,
    hidden_size=hidden_size,
    temporal_width_past=temporal_width_past,
    temporal_width_future=temporal_width_future,
    temporal_decoder_hidden=temporal_decoder_hidden,
    use_layer_norm=use_layer_norm,
    dropout=dropout,
    lr=lr,
    #likelihood=QuantileRegression(),
)

In [ ]:
pred_tide = model_tide.predict(series=val[:60], n=60, num_samples=1, past_covariates=series_cov_scaled)
hist_forecast_tide = model_tide.historical_forecasts(series=val, retrain = False, num_samples=1, past_covariates=series_cov_scaled, forecast_horizon=1)
eval_model(pred_tide, "TiDE forecast", train_set=train, val_set=val)
eval_model(hist_forecast_tide, "TiDE historical forecast", train_set=train, val_set=val)


In [ ]:

for i in range(20):
    plt.figure(figsize=(15, 5))
    Scaler_data.inverse_transform(train).univariate_component(i).plot()
    #Scaler_data.inverse_transform(lr_preds).univariate_component(i).plot(label="LR forecast")
    Scaler_data.inverse_transform(historical_forecasts_lr).univariate_component(i).plot(label="LR historical forecast")
    #Scaler_data.inverse_transform(pred_tide).univariate_component(i).plot(label="TiDE forecast")
    Scaler_data.inverse_transform(hist_forecast_tide).univariate_component(i).plot(label="tide historical forecast")
    Scaler_data.inverse_transform(val).univariate_component(i).plot(label="actual")

In [ ]:
in_len = 60
out_len = 2
output_chunk_shift = 0
hidden_size = 256
ff_size = 256
num_blocks = 8
activation = 'ReLU'
dropout = 0.08
norm_type = 'TimeBatchNorm2d'
normalize_before = True
lr = 1e-3

# Build and fit the model
model_tsmixer = build_fit_tsmixer_model(
    in_len=in_len,
    out_len=out_len,
    output_chunk_shift=output_chunk_shift,
    hidden_size=hidden_size,
    ff_size=ff_size,
    num_blocks=num_blocks,
    activation=activation,
    dropout=dropout,
    norm_type=norm_type,
    normalize_before=normalize_before,
    lr=lr,
    #likelihood=QuantileRegression(),
)

In [ ]:
pred_tmixer = model_tsmixer.predict(series=train, n=val_len, 
                                    num_samples=1)
eval_model(pred_tmixer, "TSMixer", train_set=train, val_set=val)

In [ ]:


for i in range(10):
    plt.figure(figsize=(15, 5))
    Scaler_data.inverse_transform(train).univariate_component(i).plot()
    Scaler_data.inverse_transform(val).univariate_component(i).plot(label="actual")
    Scaler_data.inverse_transform(pred_tmixer).univariate_component(i).plot(label="forecast")
    Scaler_data.inverse_transform(lr_preds).univariate_component(i).plot(label="LR forecast")

In [ ]:
#parameter optimization, this does not work...
from ray import tune
from ray.tune import CLIReporter
from ray.tune.integration.pytorch_lightning import TuneReportCheckpointCallback
from ray.tune.schedulers import ASHAScheduler
from torchmetrics import MeanAbsoluteError, MeanAbsolutePercentageError, MetricCollection

def train_model_tsmix(model_args, callbacks):
    torch_metrics = MetricCollection([MeanAbsolutePercentageError(), MeanAbsoluteError()])
    # Create the model using model_args from Ray Tune
    model = TSMixerModel(
        input_chunk_length=60,
        output_chunk_length=30,
        n_epochs=100,
        torch_metrics=torch_metrics,
        pl_trainer_kwargs={"callbacks": callbacks, "enable_progress_bar": True},
        **model_args)

    model.fit(
        series=train,
        val_series=val,
        past_covariates=series_cov_scaled,
        val_past_covariates=series_cov_scaled,
    )

my_stopper = EarlyStopping(
    monitor="val_MeanAbsolutePercentageError",
    patience=5,
    min_delta=0.00,
    mode='min',
)
# set up ray tune callback
tune_callback = TuneReportCheckpointCallback(
    {
        "loss": "val_loss",
        "MAPE": "val_MeanAbsolutePercentageError",
    },
    on="validation_end",
)

config = {
    "hidden_size": tune.choice([64, 128]),
    "ff_size": tune.choice([64, 128]),
    "num_blocks": tune.choice([2, 4, 6]),
    "dropout": tune.uniform(0, 0.3),
    "normalize_before": tune.choice([True, False]),
}

reporter = CLIReporter(
    parameter_columns=list(config.keys()),
    metric_columns=["loss", "MAPE", "training_iteration"],
)

resources_per_trial = {"cpu": 1, "gpu": 2}

num_samples = 10

scheduler = ASHAScheduler(max_t=1000, grace_period=3, reduction_factor=2)

train_fn_with_parameters = tune.with_parameters(
    train_model_tsmix, callbacks=[my_stopper, tune_callback],
)

analysis = tune.run(
    train_fn_with_parameters,
    resources_per_trial=resources_per_trial,
    # Using a metric instead of loss allows for
    # comparison between different likelihood or loss functions.
    metric="MAPE",  # any value in TuneReportCallback.
    mode="min",
    config=config,
    num_samples=num_samples,
    scheduler=scheduler,
    progress_reporter=reporter,
    name="tune_darts",
)

print("Best hyperparameters found were: ", analysis.best_config)


In [ ]:
#anomaly detection
from darts.ad import (
    ForecastingAnomalyModel,
    KMeansScorer,
    NormScorer,
    WassersteinScorer,

)
from darts.ad.utils import (
    eval_metric_from_binary_prediction,
    eval_metric_from_scores,
    show_anomalies_from_scores,
)
anomaly_model_lr = ForecastingAnomalyModel(
    model=lr_model,
    scorer=[
        NormScorer(ord=1, component_wise=True),
        WassersteinScorer(window=2, window_agg=True, component_wise=True),
    ],
)

START = 0.1
anomaly_model_lr.fit(train, start=START, allow_model_training=False, verbose=True)

anomaly_scores, model_forecasting = anomaly_model_lr.score(val, start=START, verbose=True, return_model_prediction=True, forecast_horizon=3)
pred_start = model_forecasting.start_time()

model_forecasting.univariate_component(0).plot()
val.univariate_component(0).plot()




In [ ]:
windows = [1, 10]
scorer_names = [f"{scorer}_{w}" for scorer, w in zip(anomaly_model_lr.scorers, windows)]
show_anomalies_from_scores(
    series=val,
    pred_scores=anomaly_scores,
    pred_series=model_forecasting,
    window=windows,
    title="Anomaly results using a forecasting method",
    names_of_scorers=scorer_names,
)
